In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from utils import *
from exp_utils import *
from tests import *
from cd_models import *

####### Fixing Seed #######
random_seed=42
np.random.seed(random_seed)
###########################

In [2]:
values = np.round(np.arange(-.12,.12,.01)+.005,3)
print(values)

[-0.115 -0.105 -0.095 -0.085 -0.075 -0.065 -0.055 -0.045 -0.035 -0.025
 -0.015 -0.005  0.005  0.015  0.025  0.035  0.045  0.055  0.065  0.075
  0.085  0.095  0.105  0.115]


In [3]:
d=3
n = 5000
B=100
n_p=100 #number of simulations used to estimate power
alpha=.05
task='class'
test=.5

In [4]:
pvals=[]

for k in tqdm(range(n_p)):
    for gamma in values:
        for delta in values:

            ### Setting-up data
            Xs, ys, Xt, yt = GenData2(gamma, delta, d=d, n=n)
            Xs_train, Xs_test, ys_train, ys_test, Zs_train, Zs_test, \
            Xt_train, Xt_test, yt_train, yt_test, Zt_train, Zt_test = prep_data(Xs, ys, Xt, yt, test=test, task=task, random_state=random_seed)            
            
            ###
            totshift_model = KL(boost=False, cv=None)
            totshift_model.fit(Zs_train, Zt_train)
            
            ###            
            tot=Permut(Zs_test, Zt_test, totshift_model, B=B, verbose=False)
            lab=PermutDiscrete(ys_test, yt_test, B=B, verbose=False)
            conc1=LocalPermut(Xs_test, ys_test, Xt_test, yt_test, 
                              totshift_model, labshift_model=None, task=task, B=B, verbose=False)

            ###
            pvals.append([gamma,delta,tot['pval'],conc1['pval'],lab['pval']])

100%|██████████████████████████████████████| 100/100 [9:27:52<00:00, 340.72s/it]


In [5]:
pvals=np.array(pvals)

pvals.shape

(57600, 5)

In [6]:
M=np.zeros((3,len(values),len(values)))
i=0

for gamma in values:
    j=0
    for delta in values:
        ind=np.min(pvals[:,:2]==[gamma,delta], axis=1)
        M[:,len(values)-i-1,j]=np.mean(pvals[ind,2:]<alpha, axis=0)
        j+=1
    i+=1

np.save('outputs/prior_conc.npy', M)